# Imports

In [ ]:
import numpy as np
import ipyvolume.pylab as p3
import pandas as pd
import io
import os
import subprocess as sp
import IPython
import ipyvolume as ipv
import itertools as it
import ipywidgets as ipw
import time
from datetime import datetime
import lammps_gen as lg
import IPython.core.display as disp
import requests
from concurrent.futures import ThreadPoolExecutor
import json

%matplotlib inline

# IPyWidgets Test

In [ ]:
# ipywidgets test
ipw.interact(lambda x: x, x=True)

# NEWT Rest API Auth

In [ ]:
class NEWTAuthWidget(ipw.VBox):
    def __init__(self):
        # Call VBox constructor
        super(NEWTAuthWidget, self).__init__()
        
        # Variables
        title_html = """\
        <h3>NEWT Authenticator</h3>
        """
        
        # Define widgets
        self._title_widget = ipw.HTML(value=title_html)
        self._user_input = ipw.Text(description='Username')
        self._password_input = ipw.Password(description='Password')
        self._login_button = ipw.Button(description='Login')
        self._logout_button = ipw.Button(description='Logout', disabled=True)
        self._results_box = ipw.Output()
        
        # NEWT Session object
        self._session = requests.Session()
            
        # Define layout
        self.children = [
            self._title_widget,
            ipw.HBox([
                ipw.VBox([
                    self._user_input,
                    self._password_input,
                ]),
                ipw.VBox([
                    self._login_button,
                    self._logout_button
                ])
            ]),
            self._results_box
        ]
        
        # Logical group of login UI elements
        self._login_elements = [
            self._user_input,
            self._password_input,
            self._login_button
        ]
        
        # Connect UI elements to logic functions
        self._user_input.on_submit(self._login)
        self._password_input.on_submit(self._login)
        self._login_button.on_click(self._login)
        
        self._logout_button.on_click(self._logout)
                
        
    def _login(self, caller):
        "To be called by UI elements. Request authorization, return request object, and reset password field."
        
        # Get username and password
        username = self._user_input.value
        password = self._password_input.value
        
        # Reset password
        self._password_input.value = ''
        
        # Disable further login
        self._disable_login()
        
        # Submit authorization request
        request = self._authorize(self._session, username, password)
        
        # Verify auth request
        request_success = self._verify_auth_request(request)
        
        if request_success:
            # Enable logout
            self._enable_logout()
        else:
            # Reopen login form
            self._enable_login()
        
        # Update results box
        self._display_request(request, request_type='login')
    
    def _logout(self, caller):
        "Exit current NEWT auth session"
        
        # Submit logout request
        request = self._deauthorize(self._session)
        
        # Update results box
        self._display_request(request, request_type='logout')
        
        # Return GUI to login state
        self._disable_logout()
        self._enable_login()
         
    def _verify_auth_request(self, request):
        "Determine whether authentication request was successful"
        
        return request.json()['auth']
    
    def check_auth(self):
        "Determine whether session is authenticated."
        
        return self._verify_auth_request(
            self._session.get("https://newt.nersc.gov/newt/auth")
        )
        
    def _disable_login(self):
        "Disable login form"
        
        for el in self._login_elements:
            el.disabled = True
        
    def _enable_login(self):
        "Enable login form"
        
        for el in self._login_elements:
            el.disabled = False
        
    def _disable_logout(self):
        "Disable logout button"
        
        self._logout_button.disabled = True
        
    def _enable_logout(self):
        "Enable logout button"
        
        self._logout_button.disabled = False
        
    def _display_request(self, request, request_type):
        "Update results box to display result of requested login or logout."
        
        self._results_box.clear_output()
        
        with self._results_box:
            if request_type == 'login':
                self._render_login_status(request)
            elif request_type == 'logout':
                self._render_logout_status(request)
            else:
                raise ValueError("Unknown NEWT request type")
            self._render_auth_table(request)
    
    def _request_json_to_html(self, request):
        "Convert json (python dict) of request to a nice HTML format"
        
        df = pd.DataFrame(list(request.json().items()))
        return df.to_html(index=False, header=False)
    
    def _render_login_status(self, request):
        "Write success/failure message to the results_box."
        
        status = self._verify_auth_request(request)
        
        if status:
            result = "Success!"
            msg_type = "success"
        else:
            result = "Failure."
            # msg_type = "warning"
            msg_type = "danger"
            
        html_output = """\
        <div class="alert alert-{msg_type}">
        Login {result}
        </div>
        """.format(
            result=result,
            msg_type=msg_type
        )
        
        with self._results_box:
            disp.display(
                disp.HTML(
                    html_output
                )
            )
    def _render_logout_status(self, request):
        "Write success/failure message to the results_box."
        
        status = self._verify_auth_request(request)
        
        if not status:
            result = "Success!"
            msg_type = "info"
        else:
            result = "Failure."
            msg_type = "danger"
            
        html_output = """\
        <div class="alert alert-{msg_type}">
        Logout {result}
        </div>
        """.format(
            result=result,
            msg_type=msg_type
        )
        
        with self._results_box:
            disp.display(
                disp.HTML(
                    html_output
                )
            )
            
    def _render_auth_table(self, request):
        disp.display(disp.HTML(
                self._request_json_to_html(request)
            ))
        
    def _authorize(self, session, username, password):
        "Send username & password, and request authorization"
        
        return session.post("https://newt.nersc.gov/newt/auth", {"username": username, "password": password})
        
        
    def _deauthorize(self, session):
        "Send logout request"
        
        return session.get("https://newt.nersc.gov/newt/logout")

---
# SLURM Queue Widget

In [ ]:
class QueueWidget(ipw.VBox):
    "Widget to show SLURM queue, allowing search by user."
    
    def __init__(self, auth_widget, user='oevans'):
        "QueueWidget constructor."
        # Call VBox constuctor
        super(QueueWidget, self).__init__()
        
        # Variables
        self._refresh_text = "Refresh"
        self._loading_text = "Loading..."
        self._current_queue = pd.DataFrame()
        self._poll_delay = 5
        
        # Authenticated NEWTAuthWidget
        self._auth_widget = auth_widget
        
        # Define GUI elements
        self._user_input = ipw.Text(description='User', value=user)
        self._output_area = ipw.Output()
        self._refresh_button = ipw.Button(description=self._refresh_text)
        self._watch_checkbox = ipw.Checkbox(description='Watch?', value=False)
        # Seems to crash kernel - disable for now.
        self._watch_checkbox.disabled = True
        
        # Elements which should be disabled when GUI is disabled
        self._ui_elements = [
            self._user_input,
            self._refresh_button,
            self._watch_checkbox
        ]
        self._ui_disabled_states = [el.disabled for el in self._ui_elements]
        
        # Also create a log viewer which is not attached to the widget
        self._log = ipw.Output()
        
        # Horizontal container for refresh button and checkbox
        self._bottom_container = ipw.HBox([
            self._refresh_button,
            self._watch_checkbox
        ])

        # Define layout
        self.children = [
            self._user_input,
            self._output_area,
            self._bottom_container
        ]
        
        # Connnect GUI to model
        self._user_input.on_submit(self._disable_and_refresh)
        self._refresh_button.on_click(self._disable_and_refresh)
        self._watch_checkbox.observe(self._react_to_watch_checkbox, names='value')
        
        # Search queue with default user
        self._refresh_queue()
    
    def _disable_gui(self):
        "Disable GUI elements while loading."
        
        for el_num, el in enumerate(self._ui_elements):
            # Save current disabled state to revert to
            # (don't want to re-enable a widget which
            # should be disabled)
            self._ui_disabled_states[el_num] = el.disabled
            el.disabled = True
            
        self._refresh_button.description = self._loading_text
        
    def _enable_gui(self):
        "Enable GUI elements after loading."
        
        for el_num, el in enumerate(self._ui_elements):
            el.disabled = self._ui_disabled_states[el_num]
            
        self._refresh_button.description = self._refresh_text
  
    def _disable_and_refresh(self, caller=None):
        "Refresh SLURM squeue and disable GUI. Called by text submit or button click."
        
        # Disable GUI while queue is loading
        self._disable_gui()
        
        self._refresh_queue()

        # Enable GUI after table is displayed
        self._enable_gui()
    
    def _refresh_queue(self):
        "Refresh SLURM squeue display."
        
        with self._log:
            print("Polling now..")
        
        # Retreive queue data 
        # queue_df = QueueWidget.squeue(self._user_input.value)
        
        session = self._auth_widget._session
        
        query_filter = dict(
            user = self._user_input.value
        )
        
        return_fields = [
            'user',
            'status',
            #'repo',
            #'rank_bf',
            'qos',
            'name',
            'timeuse',
            #'source',
            'hostname',
            'jobid',
            'queue',
            'submittime',
            #'reason',
            'memory',
            'nodes',
            'timereq',
            'procs',
            #'rank_p'
        ]
        
        queue_df = self.newt_queue(session, query_filter, return_fields)
        
        # If request failed, stop trying to display
        if queue_df is None:
            return
        
        # Determine whether new queue is different than previous
        # Comparing dataframes returns a boolean dataframe
        # .all().all() reduces boolean DataFrame along both dimensions
        # to a single boolean value.
        # Using not outside instead of != inside to allow for empty DF == empty DF
        try:
            new_data = not (queue_df == self._current_queue).all().all()
        
        # If dataframes have different column headers, they aren't comparable,
        # and a ValueError will be raised, in which case they aren't the same.
        except ValueError:
            new_data = True
            
        # Redraw table if new data is present
        if new_data:
            with self._log:
                print("New data!")
            self._output_area.clear_output()
            with self._output_area:
                disp.display(queue_df)
        else:
            with self._log:
                print("No new data.")

        self._current_queue = queue_df
    
    def _watch_queue(self):
        "Refresh queue every `self._poll_delay` seconds."
        
        while self._watch_checkbox.value:
            self._refresh_queue()
            time.sleep(self._poll_delay)
        
        # Log end of polling
        with self._log:
            print("Ending polling.\n")
            
        
    def _react_to_watch_checkbox(self, change):
        "Start polling queue when checkbox is enabled"
        
        if change['new']:
            with self._log:
                print("Beginning polling.")
            self._watch_queue()
        
    def newt_queue(self, session, query_filter, return_fields, machine='cori'):
        """
        Use NEWT to query specified fields from an authenticated session, filtered as desired.
        session - authenticated requests session
        query_filter - dict of search fields and values to filter results (e.g. username, date)
        return_fields - fields to return for each job 
        machine - NERSC machine to query
        
        returns a pandas DataFrame
        """
    
        base_url = "https://newt.nersc.gov/newt/"
        url_template = base_url + "queue/{machine}?{query_filter}"
        query_filter = '&'.join(['{}={}'.format(*item) for item in query_filter.items() if item[1] != ''])
        
        full_url = url_template.format(
            machine=machine,
            query_filter=query_filter
        )
        
        request = session.get(full_url)
        
        try:
            jobs = request.json()
        
            if self._auth_widget.check_auth():
                return pd.DataFrame(
                    data=[[job[key] for key in return_fields] for job in jobs],
                    columns=return_fields
                )
            else:
                # Display not authenticated warning
                
                html_output = """\
                <div class="alert alert-warning">
                Not authenticated.
                </div>
                """

                self._output_area.clear_output()
                
                with self._output_area:
                    disp.display(
                        disp.HTML(
                            html_output
                        )
                    )

        
        except ValueError:
            self._display_error(request)
            
    
    def _display_error(self, request):
        self._output_area.clear_output()
        with self._output_area:
            disp.display(disp.HTML(request.text))
    
    @staticmethod
    def squeue(user=''):
        "Query SLURM squeue for jobs by a particular user as pandas DataFrame."
        
        if len(user.split()) < 1:
            cmd_str = 'squeue'
        else:
            cmd_str = 'squeue -u {user}'.format(user=user)
        table_string, err = QueueWidget.shell(cmd_str)

        if err is not None:
            raise OSError(err)

        table_lists = [l.split() for l in table_string.split('\n')[:-1] if len(l.split()) == 8]
        table_header = table_lists[0]
        table_data = table_lists[1:]

        return pd.DataFrame(table_data, columns=table_header)
    
    @staticmethod
    def shell(cmd_str):
        "Run system command and return (sterr, stdout) as tuple of strings."
        
        cmd_list = cmd_str.split()
        proc = sp.Popen(cmd_list, stdout=sp.PIPE)
        
        # Convert stdout and stderr from bytes to str
        stdout, stderr = (b.decode('utf-8') if type(b) is bytes else None for b in proc.communicate())
        
        return stdout, stderr

---

# LAMMPS Graphene Workflow

## Graphene Sheet Widget

In [ ]:
class GrapheneSheetWidget(ipw.VBox):
    "Widget to specify parameters for an individual graphene sheet."
    
    def __init__(self, config=None):
        super(GrapheneSheetWidget, self).__init__()
        
        # Variables
        self.loc_x_range = (-10, 10)
        self.loc_y_range = (-10, 10)
        self.loc_z_range = (-10, 10)
        
        maxhex = 100
        
        self.bond_length_range = (1, 10)
        self.pair_length_range = (1, 10)
        
        self.bond_strength_expnt_range = (-5, 5)
        self.angle_strength_expnt_range = (-5, 5)
        self.pair_strength_expnt_range = (-5, 5)
        
        slider_style = dict(
            description_width = 'initial'
        )
        slider_layout = dict(
            max_width = '250px'
        )
        
        # Define UI elements
        loc_header = ipw.HBox(
            children=[ipw.HTML(
                "<strong>Location</strong>"
            )],
            layout=ipw.Layout(
                justify_content='flex-start'
            )
        )
        self._loc_x_slider = ipw.FloatSlider(
            min=self.loc_x_range[0], 
            max=self.loc_x_range[1], 
            description='x',
            style=slider_style,
            layout=slider_layout
        )
        self._loc_y_slider = ipw.FloatSlider(
            min=self.loc_y_range[0], 
            max=self.loc_y_range[1], 
            description='y',
            style=slider_style,
            layout=slider_layout
        )
        self._loc_z_slider = ipw.FloatSlider(
            min=self.loc_z_range[0], 
            max=self.loc_z_range[1], 
            description='z',
            style=slider_style,
            layout=slider_layout
        )
        
        num_header = ipw.HBox(
            children=[ipw.HTML(
                "<strong>Number of Hexagons</strong>"
            )],
            layout=ipw.Layout(
                justify_content='flex-start'
            )
        )
        # These must be odd numbers, hence step=2
        self._hex_x_slider = ipw.IntSlider(
            min=1,
            max=maxhex, 
            step=2,
            description='x',
            style=slider_style,
            layout=slider_layout
        )
        self._hex_y_slider = ipw.IntSlider(
            min=1, 
            max=maxhex,
            step=2,
            description='y',
            style=slider_style,
            layout=slider_layout
        )
        
        length_header = ipw.HBox(
            children=[ipw.HTML(
                "<strong>Potential Lengths</strong>"
            )],
            layout=ipw.Layout(
                justify_content='flex-start'
            )
        )
        self._bond_length_slider = ipw.FloatSlider(
            min=self.bond_length_range[0],
            max=self.bond_length_range[1],
            description='Bond',
            style=slider_style,
            layout=slider_layout
        )
        self._pair_length_slider = ipw.FloatSlider(
            min=self.pair_length_range[0],
            max=self.pair_length_range[1],
            description='Pair',
            style=slider_style,
            layout=slider_layout
        )
        
        strength_header = ipw.HBox(
            children=[ipw.HTML(
                "<strong>Potential Strengths Expnts.</strong>"
            )],
            layout=ipw.Layout(
                justify_content='flex-start'
            )
        )
        self._bond_strength_expnt_slider = ipw.FloatSlider(
            min=self.bond_strength_expnt_range[0],
            max=self.bond_strength_expnt_range[1],
            description='Bond',
            style=slider_style,
            layout=slider_layout
        )
        self._pair_strength_expnt_slider = ipw.FloatSlider(
            min=self.pair_strength_expnt_range[0],
            max=self.pair_strength_expnt_range[1],
            description='Pair',
            style=slider_style,
            layout=slider_layout
        )
        self._angle_strength_expnt_slider = ipw.FloatSlider(
            min=self.angle_strength_expnt_range[0],
            max=self.angle_strength_expnt_range[1],
            description='Angle',
            style=slider_style,
            layout=slider_layout
        )
        
        self._reset_button = ipw.Button(
            description="Reset",
            button_style="warning"
        )
        
        # Remove from controller.
        # Connected by controller, not by this widget
        self._remove_button = ipw.Button(
            description="Remove",
            button_style="danger"
        )
        
        reset_button_container = ipw.HBox(
            children=[self._reset_button],
            layout=ipw.Layout(
                justify_content='flex-start'
            )
        )
        
        self.children = [
            loc_header,
            self._loc_x_slider,
            self._loc_y_slider,
            self._loc_z_slider,
            
            num_header,
            self._hex_x_slider,
            self._hex_y_slider,

            length_header,
            self._bond_length_slider,
            self._pair_length_slider,

            strength_header,
            self._bond_strength_expnt_slider,
            self._pair_strength_expnt_slider,
            self._angle_strength_expnt_slider,

            self._reset_button,
            self._remove_button
        ]

        self.layout = ipw.Layout(
            display='flex',
            justify_content='space-between',
            width=u"302px"
        )
        
        # Logic
        self._reset_button.on_click(self._reset_sliders)

        # Set values on creation if config dict is provided
        if config is not None:
            self.set_values(config)
    
    def set_values(self, config):
        self._loc_x_slider.value = config['loc'][0]
        self._loc_y_slider.value = config['loc'][1]
        self._loc_z_slider.value = config['loc'][2]
        loc=[
            self._loc_x_slider.value,
            self._loc_y_slider.value,
            self._loc_z_slider.value,
        ],
        self._hex_x_slider.value = config['nx']
        self._hex_y_slider.value = config['ny']
        self._bond_length_slider.value = config['bond_length']
        self._pair_length_slider.value = config['pair_length']
        self._bond_strength_expnt_slider.value = np.log10(config['bond_strength'])
        self._pair_strength_expnt_slider.value = np.log10(config['pair_strength'])
        self._angle_strength_expnt_slider.value = np.log10(config['angle_strength'])
        
    def _reset_sliders(self, caller=None):
        self._loc_x_slider.value = 0
        self._loc_y_slider.value = 0
        self._loc_z_slider.value = 0
        self._bond_length_slider.value = 1
        self._pair_length_slider.value = 1
        self._bond_strength_expnt_slider.value = 0
        self._pair_strength_expnt_slider.value = 0
        self._angle_strength_expnt_slider.value = 0
    
    def generate_sheet_dict(self):
        return dict(
            loc=[
                self._loc_x_slider.value,
                self._loc_y_slider.value,
                self._loc_z_slider.value,
            ],
            nx = self._hex_x_slider.value,
            ny = self._hex_y_slider.value,
            bond_length = self._bond_length_slider.value,
            pair_length = self._pair_length_slider.value,
            bond_strength = 10 ** self._bond_strength_expnt_slider.value,
            pair_strength = 10 ** self._pair_strength_expnt_slider.value,
            angle_strength = 10 ** self._angle_strength_expnt_slider.value,
        )
        

## Graphene Controller Widget

In [ ]:
def Space(height=0, width=0):
    return ipw.Box(
        layout=ipw.Layout(
            width=u'{}px'.format(width),
            height=u'{}px'.format(height)
        )
    )

In [ ]:
class GrapheneControllerWidget(ipw.VBox):
    
    def __init__(self, auth_widget):
        super(GrapheneControllerWidget, self).__init__()
        
        # Variables
        maxbounds = [[-100, 100],
                     [-100, 100],
                     [-100, 100]]
        
        maxprocs = 32
        maxnodes = 32
        maxsteps = 1e20
        
        slider_style = dict(
            description_width = 'initial'
        )
        slider_layout = dict(
            max_width = '250px'
        )
        
        # UI Elements
        self._title = ipw.HTML("<h1>Graphene Simulator</strong>")
        self._accordion = ipw.Accordion()
        
        self._name_label = ipw.Label("Simulation Name")
        self._name = ipw.Text()
        
        self._base_dir_label = ipw.Label("Base Directory")
        self._base_dir = ipw.Text(value=os.getcwd())
        
        self._queue_label = ipw.Label("Queue")
        self._queue = ipw.Text('cori')
        
        self._partition_label = ipw.Label("Partition")
        self._partition = ipw.Text('debug')
        
        self._job_time_label = ipw.Label("Time Allocation")
        self._job_time = ipw.Text('10:00')
        
        self._nodes_label = ipw.Label("Number of Nodes")
        self._nodes = ipw.IntSlider(
            min=1, 
            max=maxnodes,
            layout=slider_layout,
            style=slider_style
        )
        
        self._bounds_label = ipw.Label("Simulation Bounds")
        self._x_bounds_slider = ipw.IntRangeSlider(
            min=maxbounds[0][0], 
            max=maxbounds[0][1], 
            description='x',
            layout=slider_layout,
            style=slider_style
        )
        self._y_bounds_slider = ipw.IntRangeSlider(
            min=maxbounds[1][0], 
            max=maxbounds[1][1], 
            description='y',
            layout=slider_layout,
            style=slider_style
        )
        self._z_bounds_slider = ipw.IntRangeSlider(
            min=maxbounds[2][0], 
            max=maxbounds[2][1], 
            description='z',
            layout=slider_layout,
            style=slider_style
        )
        
        self._procs_label = ipw.Label("Processor Grid")
        self._x_procs_slider = ipw.IntSlider(
            min=1, 
            max=maxprocs, 
            description='x',
            layout=slider_layout,
            style=slider_style
        )
        self._y_procs_slider = ipw.IntSlider(
            min=1, 
            max=maxprocs, 
            description='y',
            layout=slider_layout,
            style=slider_style
        )
        self._z_procs_slider = ipw.IntSlider(
            min=1, 
            max=maxprocs, 
            description='z',
            layout=slider_layout,
            style=slider_style
        )
        
        self._nsteps_label = ipw.Label("Number of Timesteps")
        self._nsteps = ipw.BoundedIntText(min=1, max=maxsteps, value=10000)
        
        self._dump_freq_label = ipw.Label("Dump Frequency")
        self._dump_freq = ipw.BoundedIntText(min=1, max=maxsteps, value=1000)
        
        
        self._sheet_widget_area = ipw.HBox()
        
        self._add_button = ipw.Button(
            description='Add Sheet',
            icon='plus', 
            button_style='success'
        )
        self._simulate_button = ipw.Button(
            description="Run simulation",
            button_style='success'
        )
        
        self._status_label = ipw.Label()

        self._log_area = ipw.Output()
        
        self._save_load_header = ipw.HTML('<strong>Save/Load Widget State</strong>')
        self._config_path_label = ipw.Label('Config file path')
        self._config_input = ipw.Text()
        self._load_config_button = ipw.Button(description='Load config')
        self._save_config_button = ipw.Button(description='Save config')
        
        # Define layout
        
        self._config_box = ipw.HBox([
            ipw.VBox([
                self._name_label,
                self._name,

                Space(10),
                self._base_dir_label,
                self._base_dir,

                Space(10),
                self._queue_label,
                self._queue,

                Space(10),
                self._partition_label,
                self._partition,

                Space(10),
                self._job_time_label,
                self._job_time,

                Space(10),
                self._nsteps_label,
                self._nsteps,

                Space(10),
                self._dump_freq_label,
                self._dump_freq,

                Space(20),
                ipw.HBox([
                    self._simulate_button,
                    Space(width=10),
                    self._status_label
                ])
            ]),
            
            Space(width=50),
            
            ipw.VBox([
                self._nodes_label,
                self._nodes,

                Space(10),
                self._procs_label,
                self._x_procs_slider,
                self._y_procs_slider,
                self._z_procs_slider,

                Space(10),
                self._bounds_label,
                self._x_bounds_slider,
                self._y_bounds_slider,
                self._z_bounds_slider,
                
                Space(20),
                self._save_load_header,
                self._config_path_label,
                self._config_input,
                ipw.HBox([
                    self._load_config_button,
                    self._save_config_button
                ])
            ])
        ])
        
        self._generator_area = ipw.VBox([
            self._add_button,
            self._sheet_widget_area
        ])
        
        self._accordion.children = [
            self._config_box,
            self._generator_area
        ]
        
        self.children = [
            self._title,
            self._accordion
        ]
        
        # Accordion titles
        self._accordion.set_title(0, 'Simulation Configuration')
        self._accordion.set_title(1, 'Graphene Generator')
        
        # Additional style
        #self._add_button.layout.width = u'36px'
        
        # Connect UI to logic
        self._add_button.on_click(self._add_sheet_widget)
        self._simulate_button.on_click(self.submit_simulation)
        self._name.observe(self._update_config_path)
        self._base_dir.observe(self._update_config_path)
        
        self._load_config_button.on_click(self._load_config_wrapper)
        self._save_config_button.on_click(self._save_config_wrapper)
        
        self._auth_widget = auth_widget
        self._update_config_path()
    
    def _add_sheet_widget(self, caller=None, config=None):
        print("Creating new widget.")

        new_widget = GrapheneSheetWidget(config)
        print("Created.")
        new_widget._remove_button.on_click(self._remove_sheet_widget)
        # Associate button with sheet widget to allow for removal by button click.
        new_widget._remove_button.parent_widget = new_widget
        print("Linked.")
        
        # Add to widget area
        self._sheet_widget_area.children = self._sheet_widget_area.children + (new_widget,)
        
    def _update_config_path(self, change=None):
        name = self._name.value
        self._config_input.value = os.path.join(
            self._base_dir.value,
            os.path.join(name, name+'.config')
        )
    
    def _remove_sheet_widget(self, caller):
        with self._log_area:
            print("Removing widget:")
            print(caller)
        
        # Function to exclude parent_widget of caller (remove button)
        # from children of sheet_widget_area
        
        exclude_widget = lambda widget: id(widget) != id(caller.parent_widget)
        self._sheet_widget_area.children = tuple(filter(exclude_widget, self._sheet_widget_area.children))
        
    def submit_simulation(self, caller):
        "Submit simulation given the simulation parameters in this widget."

        with self._log_area:
            simulate_graphene(
                job_name = self._name.value,
                auth_widget = self._auth_widget,
                sheet_dict_list = [
                    sheet.generate_sheet_dict() 
                    for sheet in self._sheet_widget_area.children
                ],
                nsteps = self._nsteps.value,
                dump_freq = self._dump_freq.value,
                base_dir = self._base_dir.value,
                bounds = np.array([
                    self._x_bounds_slider.value,
                    self._y_bounds_slider.value,
                    self._z_bounds_slider.value,
                ]),
                px = self._x_procs_slider.value,
                py = self._y_procs_slider.value,
                pz = self._z_procs_slider.value,
                nodes = self._nodes.value,
                queue = self._queue.value,
                job_time = self._job_time.value,
            )

            self._status_label.value = "Job submitted!"
            print("Job submitted.")
    
    def _save_config_wrapper(self, caller):
        with self._log_area:
            self.save_config(outfile=self._config_input.value)
        
    def _load_config_wrapper(self, caller):
        with self._log_area:
            print("Load!")
            self.load_config(infile=self._config_input.value)
            
        
    def save_config(self, outfile):
        "Save present state of widget to a file."
        config = dict(
            job_name = self._name.value,
            sheet_dict_list = [
                sheet.generate_sheet_dict() 
                for sheet in self._sheet_widget_area.children
            ],
            nsteps = self._nsteps.value,
            dump_freq = self._dump_freq.value,
            base_dir = self._base_dir.value,
            bounds = list(map(
                list,
                (
                    self._x_bounds_slider.value,
                    self._y_bounds_slider.value,
                    self._z_bounds_slider.value,
                )
            )),                
            px = self._x_procs_slider.value,
            py = self._y_procs_slider.value,
            pz = self._z_procs_slider.value,
            nodes = self._nodes.value,
            queue = self._queue.value,
            job_time = self._job_time.value,
        )
    
        with open(outfile, 'w') as f:
            json.dump(config, f, indent=4)
            
        print("Config saved to {}".format(outfile))

    def load_config(self, infile):
        "Load state of widget from config"
        
        with open(infile) as f:
            config = json.load(f)
        
        self._name.value = config['job_name']
        
        # Remove existing sheet widgets
        for sheet in self._sheet_widget_area.children:
            self._remove_sheet_widget(sheet._remove_button)

        # Create loaded sheet widgets
        for sheet_dict in config['sheet_dict_list']:
            self._add_sheet_widget(config=sheet_dict)

        self._nsteps.value = config['nsteps']
        self._dump_freq.value = config['dump_freq']
        self._base_dir.value = config['base_dir']
        
        self._x_bounds_slider.value = config['bounds'][0]
        self._y_bounds_slider.value = config['bounds'][1]
        self._z_bounds_slider.value = config['bounds'][2]
        
        self._x_procs_slider.value = config['px']
        self._y_procs_slider.value = config['py']
        self._z_procs_slider.value = config['pz']
        self._nodes.value = config['nodes']
        self._queue.value = config['queue']
        self._job_time.value = config['job_time']      
        
        print("Config loaded from {}".format(infile))

In [23]:
a = NEWTAuthWidget()
a

A Jupyter Widget

username,oevans
session_lifetime,43199
auth,True
newt_sessionid,418ed9de75bfeadf234aecd9522bb3bb


In [24]:
q = QueueWidget(a)
q

A Jupyter Widget

In [25]:
c = GrapheneControllerWidget(a)
c

A Jupyter Widget

In [ ]:
c._log_area.clear_output()
c._log_area

## File Browser

In [26]:
class FileBrowserWidget(ipw.Select):
    def __init__(self):
        super(FileBrowserWidget, self).__init__()
        
        self.cwd = os.getcwd()
        self.options = self.ls()
        
        self.layout.height = u'300px'
        
        self.observe(self.perform_cd, names='index')
        
    
    def ls(self):
        full_ls = os.listdir(self.cwd)
        
        dirs = []
        files = []
        for file in full_ls:
            abspath = self.abspath(file)
            
            if os.path.isdir(abspath):
                dirs.append(file)
            else:
                files.append(file)

        # Format
        dirs = [self.format_dir(path) for path in self.sortfiles(dirs)]
        files = [self.format_file(path) for path in self.sortfiles(files)]
            
        return tuple([('.', '.'),('..', '..')] + dirs + files)
    
    def format_dir(self, path):
        return (self.dirify(path), self.abspath(path))
    
    def format_file(self, path):
        return (path, self.abspath(path))
    
    def sortfiles(self, filelist):
        "Sort files alphabetically, placing hidden files after others."
        
        hidden = []
        nonhidden = []
        
        for file in filelist:
            if file[0] == '.':
                hidden.append(file)
            else:
                nonhidden.append(file)
        
        return sorted(nonhidden) + sorted(hidden)
    
    def dirify(self, path):
        "How to represent directory names"
        return '** {} **'.format(path)
    
    def abspath(self, path):
        "Join path to cwd and return absolute path"
        return os.path.abspath(
            os.path.join(
                self.cwd,
                path
            )
        )
    
    def cd(self, path):
        self.cwd = self.abspath(path)
        self.options = self.ls()
    
    def perform_cd(self, change):
        newindex = change['new']
        newdir = self.options[newindex][1]
        if os.path.isdir(newdir):
            self.cd(newdir)

In [27]:
f = FileBrowserWidget()
f

A Jupyter Widget

## Actual Workflow

In [ ]:
def gen_lammps_data(job_name, bounds, sheet_dict_list, job_dir):
    """
    Generate graphene sheets and write in LAMMPS format.
    bounds is a 3x2 array of 3D simulation bounds
    sheet_dict_list is a list of dicts, each containing:
    - loc (3-array)
    - nx (# of hexagons in x direction)
    - ny (# of hexagons in y direction)
    - bond_length
    - pair_length
    - bond_strength
    - pair_strength
    - angle_strength
    """

    # Simulation parameters
    sim = lg.Simulation(bounds)
    
    for sheet_dict in sheet_dict_list:
        
        nx = sheet_dict['nx']
        ny = sheet_dict['ny']
        loc = sheet_dict['loc']
        bond_length = sheet_dict['bond_length']
        pair_length = sheet_dict['pair_length']
        bond_strength = sheet_dict['bond_strength']
        pair_strength = sheet_dict['pair_strength']
        angle_strength = sheet_dict['angle_strength']

        sheet = lg.GrapheneSheet(sim, nx, ny)
        
        sheet.set_loc(*loc)

        sheet.set_bond_length(bond_length)
        sheet.set_bond_strength(bond_strength)

        sheet.set_angle_measure(120)
        sheet.set_angle_strength(angle_strength)

        sheet.set_pair_length(pair_length)
        sheet.set_pair_strength(pair_strength)

    # Save
    output_loc = os.path.join(job_dir, '{job_name}.data'.format(job_name=job_name))
    sim.write(output_loc)
    
    print("LAMMPS data file written to {}.data".format(job_name))

In [ ]:
def gen_lammps_input(job_name, nsteps, dump_freq, job_dir,
    px=1, py=1, pz=1,
    lammps_template_file="/global/homes/o/oevans/graphene/lammps_template.in"):
    "Generate LAMMPS input script for graphene simulation, varying a few io parameters"
    
    # Read template file
    with open(lammps_template_file) as template_handle:
        template_str = template_handle.read()
    
    # Anonymous function to replace all parameters in string
    replace_params = lambda s: s.format(
        job_name=job_name,
        job_dir=job_dir,
        px=px,
        py=py,
        pz=pz,
        nsteps=nsteps,
        dump_freq=dump_freq
    )
    
    lammps_str = replace_params(template_str)
    
    output_name = "{}.in".format(job_name)
    
    replaced_output_name = os.path.join(job_dir, replace_params(output_name))
    
    # Write output file
    with open(replaced_output_name, 'w') as out_handle:
        out_handle.write(lammps_str)  
        
    print("LAMMPS input file written to {}".format(replaced_output_name))

In [ ]:
def gen_slurm_script(job_name, nodes=1, cores=32, partition="debug", job_time="10:00",
                    batch_template_file="/global/homes/o/oevans/graphene/batch_template",
                    job_dir="/global/homes/o/oevans/graphene/{job_name}"):
    "Generate slurm script for a job, directing output to a new folder"
    
    # Read template file
    with open(batch_template_file) as template_handle:
        template_str = template_handle.read()
        
    
    # Anonymous function to replace all parameters in string
    replace_params = lambda s: s.format(
        job_name=job_name,
        nodes=nodes,
        cores=cores,
        partition=partition,
        job_time=job_time,
        batch_template_file=batch_template_file,
        job_dir=job_dir
    )
    
    # Replace template strings
    batch_str = replace_params(template_str)
    
    output_name = "{}.batch".format(job_name)
    
    # Determine name of output batch file
    # Can handle parameter replacement
    replaced_output_name = os.path.join(job_dir, replace_params(output_name))
    
    # Write output file
    with open(replaced_output_name, 'w') as out_handle:
        out_handle.write(batch_str)
    
    
    print("SLURM batch script written to {}".format(replaced_output_name))
        

In [ ]:
def newt_submit(auth_widget, jobfile="/global/homes/o/oevans/graphene/test_job.batch", queue='cori'):
    "Submit a batch job to an HPC queue using NEWT."
    
    session = auth_widget._session
    url = "https://newt.nersc.gov/newt/queue/{queue}".format(queue=queue)
    request = session.post(url, {'jobfile': jobfile})
    
    print("Job submitted from '{}'".format(jobfile))
    #return disp.HTML(auth_widget._request_json_to_html(request))
    return request
    

In [ ]:
# Wrap all above
def simulate_graphene(job_name, auth_widget, sheet_dict_list, nsteps, 
                      dump_freq, base_dir, bounds, px=1, py=1, 
                      pz=1, nodes=1, queue="cori", partition='debug', job_time="10:00",
                      template_dir = "/global/homes/o/oevans/graphene/"):
    """Generate LAMMPS data and input files and SLURM job submission script
    for graphene simulation, and submit to SLURM queue."""
    
    # Make directory if necessary 
    job_dir = os.path.join(base_dir, job_name)
    os.makedirs(job_dir, exist_ok=True)
    
    cores = px * py * pz
    jobfile = os.path.join(job_dir, job_name+".batch")
    
    gen_lammps_data(job_name, bounds, sheet_dict_list, job_dir)
    
    gen_lammps_input(job_name, nsteps, dump_freq, 
        px=px, py=py, pz=pz, job_dir=job_dir,
        lammps_template_file=template_dir+"lammps_template.in")
    
    gen_slurm_script(job_name=job_name, nodes=nodes, cores=cores, partition=partition, job_time=job_time,
        batch_template_file=template_dir+"batch_template",job_dir=job_dir)
    
    request = newt_submit(auth_widget, jobfile=jobfile, queue=queue)
    
    print("Simulation submitted.")
    print(request.json())

Test 123

In [ ]:
b = ipw.Button()
b

In [ ]:
b.disabled=False

## Job Monitoring

In [ ]:
def monitor():
    "Monitor file for new data"
    pass

def extract_temp():
    pass

def live_2d_line():
    pass

def live_3d_scatter():
    pass

def plot_dist():
    pass

# LAMMPS Simulation

In [ ]:
class LammpsSim(object):
    def __init__(self, lammpstrj_path):
        self.lammpstrj_path = lammpstrj_path
        self.file_handle = open(lammpstrj_path, 'r')
        self.num_atoms = self.get_num_atoms()
        self.current_step_num = 0
    
    def __enter__(self):
        """
        Allows for using `with LammpsSim(...) as sim`
        Except this doesn't work if you want to continue reading the file
        after the cell is finished executing
        """
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.close()
    
    def close(self):
        try:
            self.file_handle.close()
        except:
            pass
        
    def index_timesteps(self, max_steps=100):
        """
        Read whole file, and save position of each timestep in file
        for easier seeking.
        
        Set max_steps to limit the number of timesteps read
        """
        
        # List of positions of timesteps
        self.step_index = []
        # Keep track of number of steps
        self.num_steps = 0
        
        # Seek to beginning of file
        self.file_handle.seek(0)
        
        # Loop through steps until EOF or max_steps
        for step_num in range(max_steps):
            
            # Save position
            file_pos = self.file_handle.tell()

            # Read header
            header = [self.file_handle.readline() for i in range(9)]
            # Check for EOF, in which case '' will appear in list
            # (as opposed to '\n' for a legitimate blank line)
            if '' in header:
                break
            
            # Number of atoms in this timestep
            num_atoms = int(header[3])
            for atom_num in range(num_atoms):
                # Check only for EOF
                if self.file_handle.readline() == '':
                    break
            
            # If reading atom info ended early, then the file is finished,
            # so don't try to read the next timestep
            if atom_num < num_atoms-1:
                break
                
            # If whole timestep was read sucessfully, save timestep
            self.step_index.append(file_pos)
            self.num_steps += 1
    
    def get_num_atoms(self):
        self.file_handle.seek(0)
        
        # Read first 4 lines
        # (num_atoms is on line 4)
        for line_num in range(4):
            line = self.file_handle.readline()
        
        return int(line)
    
    def seek_to_step(self, step_num):
        """
        Seek to step step_num
        Must have already run index_timesteps.
        """
        self.file_handle.seek(self.step_index[step_num])

    # Parse lammpstrj
    def read_step(self):
        """
        Read one timestep from lammps trajectory file
        Pass a Python file object pointing to the first line
        of the timestep
        (which reads 'ITEM: TIMESTEP')
        """

        # Read header - first 9 lines
        header_len = 9
        try:
            header = [
                self.file_handle.readline() 
                for x in range(header_len)
            ]
        except StopIteration as err:
            print("End of file reached.")
            return

        # Number of atoms
        num_atoms = int(header[3])
        # Box bounds
        bnds = np.array([list(map(float,header[i].split())) for i in range(5,8)]).T

        # Read data
        raw_data = [
            self.file_handle.readline()[:-1] 
            for x in range(num_atoms)
        ]

        # Convert to DataFrame
        column_headers = header[-1].split()[2:]

        string_buffer = io.StringIO('\n'.join(raw_data))
        step_data = pd.read_csv(string_buffer,
                                delimiter=' ',
                                names=column_headers,
                                index_col=False)

        # Generate actual position given scaled position and image numbers
        converted_data = self.calc_positions(step_data,num_atoms,bnds)
        return converted_data

    @staticmethod
    def calc_positions(step_data,num_atoms,bnds):
        scaled_pos = step_data.loc[:,['xs','ys','zs']].values
        im = step_data.loc[:,['ix','iy','iz']].values

        # Separate upper & lower bounds
        lo_bnd, hi_bnd = np.array(bnds)

        real_pos = (scaled_pos + im) * (hi_bnd - lo_bnd) + lo_bnd


        # Combine atom id & type with position
        combined_arr = np.hstack([
            step_data.loc[:,['id','type']].values,
            real_pos
        ])

        headers = ['id','type','x','y','z']
        converted_data = pd.DataFrame(combined_arr,
                                      columns=headers,
        )

        # Enforce appropriate dtypes
        # https://stackoverflow.com/questions/25610592/how-to-set-dtypes-by-column-in-pandas-dataframe
        dtypes = {
            'id': int,
            'type': int,
            'x': float,
            'y': float,
            'z': float
        }
        for col_name, dtype in dtypes.items():
            converted_data[col_name] = converted_data[col_name].astype(dtype)

        return converted_data
    
    @staticmethod
    def lammps_scatter(lammps_df, p3_fig):
        """
        Create 3d scatter from lammps input data
        """

        colors = ['red','green','blue','yellow',
                  'orange','purple','black','brown']
        markers = ['sphere','diamond','box','arrow']

        # All combinations of colors and markers
        # One per atom type
        cm_pairs = it.product(markers,colors)

        # Identify all atom types
        atom_types = lammps_df.loc[:,'type'].unique()
        num_types = len(atom_types)

        # Count number of scatters already plotted in figure
        num_scatters = len(p3_fig.scatters)

        # Note that LAMMPS counts atom types from 1,
        # whereas Python counts indices from 0.

        # Plot each with a different color/marker pair
        for atom_type, cm_pair in zip(atom_types, cm_pairs):
            
            print("Type {}/{}".format(atom_type, num_scatters))

            # Extract atom positions for this type
            pos = lammps_df.query('type == {}'.format(atom_type))
            # Sort atoms by id to retain identity
            # Allows for sensible visual transition between frames
            pos = pos.sort_values('id')

            # Get marker and color
            marker, color = cm_pair

            # Types which have already been plotted
            # - update plot
            if atom_type <= num_scatters:
                
                print("Updating atom type {}".format(atom_type))
                
                # Subtract one from LAMMPS atom type
                # to get Python list index
                indx = atom_type - 1

                # Extract scatter for this atom type
                sct = p3_fig.scatters[indx]

                # Update positions
                sct.x = pos['x']
                sct.y = pos['y']
                sct.z = pos['z']
                
                log_msg('test', 'Update positions for type {}'.format(atom_type))

            # Types which have not yet been plotted
            # - create new plot
            else:
                print("Creating atom type {}".format(atom_type))
                # Create a new scatter
                p3.scatter(
                    x=pos['x'], 
                    y=pos['y'], 
                    z=pos['z'],
                    marker=marker,
                    color=color
                )
    
    def update_timestep(self, step_num):
        print("Update to step {}".format(step_num))
        log_msg('test', 'Update to step {}'.format(step_num))
        self.seek_to_step(step_num)
        self.step_num = step_num
        self.lammps_df = self.read_step()
        self.lammps_scatter(self.lammps_df, self.fig)
    
    def plot_steps(self):
        """
        Main function to call to create plot & timestep widget
        Assuming static file (doesn't listen for new data)
        """
        
        self.fig = p3.figure(controls=None)
        #self.fig.camera_control = 'orbit'
        
        self.seek_to_step(0)
        lammps_df = self.read_step()
        self.lammps_scatter(lammps_df, self.fig)
        
        self.step_slider = ipw.IntSlider(
            min=0, 
            max=self.num_steps-1, 
            value=0,
            description='Timestep'
        )
        
        ipw.interactive(
            self.update_timestep, 
            step_num=self.step_slider
        )
        
        container = ipw.VBox([
            self.fig,
            self.step_slider
        ])
        
        return container
        

In [ ]:
try:
    sim.close()
except:
    pass
#sim = LammpsSim('/home/oliver/academic/research/graphene/two_sheets.lammpstrj')
sim = LammpsSim('/global/homes/o/oevans/lammps-test/graphene/two_sheets.lammpstrj')
sim.index_timesteps()
sim.plot_steps()

In [ ]:
# Test that seek_to_step is working properly
@ipw.interact(step_num=(0,100))
def test_seek(step_num):
    sim.seek_to_step(step_num)
    header = [sim.file_handle.readline() for i in range(9)]
    
    print('Step = {}'.format(step_num))
    print()
    
    print(''.join(header))
    

In [ ]:
fname = "/home/oliver/academic/research/graphene/two_sheets_ripple.lammpstrj"
with open(fname) as file_handle:
    for i in range(11):
        # Read 10 steps at a time
        for j in range(10):
            lammps_df = read_step(file_handle)
        lammps_scatter(lammps_df, fig)
        time.sleep(1)

In [ ]:
fig = p3.figure(controls=False)

def update_timestep(file_handle,step_num):
    read_step_num(file_handle,step_num)

ipw.VBox([
    fig,
    
])

In [ ]:
@ipw.interact(size=1.0)
def update_size(size):
    for sct in fig.scatters:
        sct.size = size

In [ ]:
import os

import ipywidgets as widgets


class FileBrowser(object):
    def __init__(self):
        self.path = os.getcwd()
        self._update_files()
        
    def _update_files(self):
        self.files = list()
        self.dirs = list()
        if(os.path.isdir(self.path)):
            for f in os.listdir(self.path):
                ff = self.path + "/" + f
                if os.path.isdir(ff):
                    self.dirs.append(f)
                else:
                    self.files.append(f)
        
    def widget(self):
        box = widgets.VBox()
        self._update(box)
        return box
    
    def _update(self, box):
        
        def on_click(b):
            if b.description == '..':
                self.path = os.path.split(self.path)[0]
            else:
                self.path = self.path + "/" + b.description
            self._update_files()
            self._update(box)
        
        buttons = []
        if self.files:
            button = widgets.Button(description='..', background_color='#d0d0ff')
            button.on_click(on_click)
            buttons.append(button)
        for f in self.dirs:
            button = widgets.Button(description=f, background_color='#d0d0ff')
            button.on_click(on_click)
            buttons.append(button)
        for f in self.files:
            button = widgets.Button(description=f)
            button.on_click(on_click)
            buttons.append(button)
        box.children = tuple([widgets.HTML("<h2>%s</h2>" % (self.path,))] + buttons)